This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

zLD6uzUEYs9N7GpySywv


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
def FSE_AFX_Pull(start_date = "2014-01-03", end_date = "2014-01-03"):
    p = {"start_date": start_date, "end_date": end_date, "api_key": API_KEY}
    url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json" 
    r = requests.get(url, params = p)
    return r
    
r = FSE_AFX_Pull()
print(r)

<Response [200]>


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-01-03',
  'end_date': '2014-01-03',
  'frequency': 'daily',
  'data': [['2014-01-03',
    24.04,
    24.47,
    24.03,
    24.47,
    None,
    20503.0,
    499475.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
r_2017 = FSE_AFX_Pull("2017-01-01","2017-12-31")
r_2017_dict = r_2017.json()

In [6]:
daily_open = [day[:2] for day in r_2017_dict['dataset_data']['data'] if day[1] != None]
max_open = max(daily_open, key=lambda x: x[1])
min_open = min(daily_open, key=lambda x: x[1])
print(f"Highest opening is {max_open} and lowest opening is {min_open}")

Highest opening is ['2017-12-14', 53.11] and lowest opening is ['2017-01-24', 34.0]


In [21]:
daily_change = [[day[0], round(day[2] - day[3],2)] for day in r_2017_dict['dataset_data']['data'] if day[1] != None]
max_change = max(daily_change, key=lambda x: x[1])
print(f"Largest change in any day is {max_change}")

Largest change in any day is ['2017-05-11', 2.81]


In [20]:
date_list = [day[0] for day in r_2017_dict['dataset_data']['data'] if day[1] != None] #dates descending
closing_list = [day[4] for day in r_2017_dict['dataset_data']['data'] if day[1] != None] 
days = len(closing_list)

closing_change = [round(c-c_prev,2) for c, c_prev in zip(closing_list, closing_list[1:days])]
closing_change_list = list(zip(date_list, closing_change))
max_closing_change = max(closing_change_list, key=lambda x: x[1])

print(f"Largest change between two days closings is {max_closing_change}")

Largest change between two days closings is ('2017-05-11', 1.72)


In [9]:
volume_list = [day[6] for day in r_2017_dict['dataset_data']['data']]
tot_vol = sum(volume_list)
days = len(volume_list)+1
mean_vol = tot_vol / days

print(f"Average daily trading volume during the year was {mean_vol}")

Average daily trading volume during the year was 88776.1953125


***
**Alternative approach:** Create dictionary with column names as keys and data stored as a list for each column name.
***

In [10]:
#Create dictionary
keys = r_2017_dict['dataset_data']['column_names']
values = []

for x in range(len(keys)):
    values.append([data[x] for data in r_2017_dict['dataset_data']['data']])
    
AFX_2017 = dict(zip(keys,values))

In [11]:
#Check null values
dropped_values = []
for x in range(len(keys)):
    dropped_values.append([data[x] for data in r_2017_dict['dataset_data']['data'] if data[1] == None])

AFX_2017_dropped = dict(zip(keys,dropped_values))

dropped_index = [idx for idx, x in enumerate(AFX_2017['Date']) if x in AFX_2017_dropped['Date']]

for idx in dropped_index:
    for x in AFX_2017.keys():
        print(x, AFX_2017[x][idx])
    print("*****")

Date 2017-05-01
Open None
High 42.245
Low 41.655
Close 41.72
Change -0.44
Traded Volume 86348.0
Turnover 3606589.0
Last Price of the Day None
Daily Traded Units None
Daily Turnover None
*****
Date 2017-04-17
Open None
High 42.48
Low 41.985
Close 42.2
Change None
Traded Volume 88416.0
Turnover 3734717.0
Last Price of the Day None
Daily Traded Units None
Daily Turnover None
*****
Date 2017-04-14
Open None
High 42.48
Low 41.985
Close 42.2
Change None
Traded Volume 88416.0
Turnover 3734717.0
Last Price of the Day None
Daily Traded Units None
Daily Turnover None
*****


In [12]:
#replace null opening values with closing values from previous day
values[1] = [values[4][idx+1] if x == None else x for idx, x in enumerate(values[1])]     
    
AFX_2017 = dict(zip(keys,values))

In [13]:
#Check updated values

dropped_index_update = []
for x in dropped_index:
    dropped_index_update.extend([x, x+1])

print(dropped_index_update)

AFX_2017_short = {key: AFX_2017[key] for key in ['Date', 'Open', 'Close']}

for idx in dropped_index_update:
    print("*****")
    for x in AFX_2017_short.keys():
        print(x, AFX_2017[x][idx])

[169, 170, 179, 180, 180, 181]
*****
Date 2017-05-01
Open 41.72
Close 41.72
*****
Date 2017-04-28
Open 42.17
Close 41.72
*****
Date 2017-04-17
Open 42.2
Close 42.2
*****
Date 2017-04-14
Open 42.2
Close 42.2
*****
Date 2017-04-14
Open 42.2
Close 42.2
*****
Date 2017-04-13
Open 42.06
Close 42.2


In [14]:
#Calculate highest and lowest opening prices
max_o = max(AFX_2017['Open'])
max_o_i = AFX_2017['Open'].index(max_o)
max_o_date = AFX_2017['Date'][max_o_i]

min_o = min(AFX_2017['Open'])
min_o_i = AFX_2017['Open'].index(min_o)
min_o_date = AFX_2017['Date'][min_o_i]

In [15]:
AFX_2017['High'][0] 

51.94

In [16]:
#Calculate largest change in any one day

change_Rx_list = []
for count in range(len(AFX_2017['Date'])):
    change_Rx_list.append(round(AFX_2017['High'][count] - AFX_2017['Low'][count],2))

AFX_2017['Change_Rx'] = change_Rx_list
    
max_chg = max(AFX_2017['Change_Rx'])
max_chg_i = AFX_2017['Change_Rx'].index(max_chg)
max_chg_date = AFX_2017['Date'][max_chg_i]

In [17]:
#Calculate largest change between two closings

change_c_list = []
for count in range(len(AFX_2017['Date'])-1):
    change_c_list.append(round(AFX_2017['Close'][count] - AFX_2017['Close'][count+1],2))

AFX_2017['Change_c'] = change_c_list
    
max_chg_c = max(AFX_2017['Change_c'])
max_chg_c_i = AFX_2017['Change_c'].index(max_chg_c)
max_chg_c_date = AFX_2017['Date'][max_chg_c_i]

In [18]:
#Calculate average daily trading volume during the year
days_alt = len(AFX_2017['Traded Volume'])+1
tot_vol_alt = sum(AFX_2017['Traded Volume'])
mean_vol_alt = tot_vol_alt / days_alt


med_day_even = int(days_alt/2 - 1)
med_day_low = int(days_alt/2 - 2)
med_day_high = int(days_alt/2 - 1)
sorted_vol = list(AFX_2017['Traded Volume'])
sorted_vol.sort()

if days_alt % 2 == 0:
    med_vol_alt = (sorted_vol[med_day_high] + sorted_vol[med_day_low]) / 2
else:
    med_vol_alt = sorted_vol[med_day_even]

In [19]:
#Results
print(
    f"The max open was {max_o} on {max_o_date}.\n"
    f"The min open was {min_o} on {min_o_date}.\n"
    f"The largest daily change was {max_chg} on {max_chg_date}.\n"
    f"The largest change in closing between two days was on {max_chg_c} on {max_chg_c_date}.\n"
    f"The mean daily trade volume was {mean_vol:,.0f}.\n"
    f"The median daily trade volume was {med_vol_alt:,.0f}.\n"
)

The max open was 53.11 on 2017-12-14.
The min open was 34.0 on 2017-01-24.
The largest daily change was 2.81 on 2017-05-11.
The largest change in closing between two days was on 1.72 on 2017-05-11.
The mean daily trade volume was 88,776.
The median daily trade volume was 75,630.

